In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        print(filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tsp532/att532.tsp
att532.tsp
/kaggle/input/dsj1000/dsj1000.tsp
dsj1000.tsp
/kaggle/input/data1291/d1291.tsp
d1291.tsp


In [3]:
def read_cities(filename):
    with open(filename, 'r') as f:
        # Skip the header lines
        for i in range(6):
            next(f)
        
        # Extract the coordinates and build the tuples
        cities = []
        for line in f:
            if line.startswith('EOF'):
                break
            values = line.split()
            if len(values) >= 3:
                x, y = values[1], values[2]
                cities.append((float(x), float(y)))
    return cities

In [4]:
cities = read_cities("/kaggle/input/tsp532/att532.tsp")
len(cities)

532

In [5]:
import numpy as np
import pandas as pd
import random
import time
from multiprocessing import Pool

# Define the fitness function
def fitness(individual, distances):
    distance = 0
    for i in range(len(individual)-1):
        distance += distances[individual[i], individual[i+1]]
    distance += distances[individual[-1], individual[0]]
    return distance

# Define the crossover function
def crossover(parent1, parent2):
    child = [-1] * len(parent1)
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    for i in range(startGene, endGene):
        child[i] = parent1[i]
    for i in range(len(parent2)):
        if parent2[i] not in child:
            for j in range(len(child)):
                if child[j] == -1:
                    child[j] = parent2[i]
                    break
    return child

# Define the mutation function
def mutation(individual):
    geneA = int(random.random() * len(individual))
    geneB = int(random.random() * len(individual))
    individual[geneA], individual[geneB] = individual[geneB], individual[geneA]
    return individual

# Define the genetic algorithm function
def genetic_algorithm(distances, num_processes, population_size, generations=100):
    # Create the initial population
    population = []
    for i in range(population_size):
        individual = list(range(len(distances)))
        random.shuffle(individual)
        population.append(individual)
    
    # Evolve the population
    for generation in range(generations):
        # Evaluate the fitness of each individual
        fitness_values = []
        for individual in population:
            fitness_values.append(fitness(individual, distances))
        
        # Select the parents for the next generation
        parents = []
        for i in range(population_size):
            parent1 = population[random.randint(0, population_size-1)]
            parent2 = population[random.randint(0, population_size-1)]
            if fitness(parent1, distances) < fitness(parent2, distances):
                parents.append(parent1)
            else:
                parents.append(parent2)
        
        # Create the next generation
        with Pool(num_processes) as pool:
            population = pool.starmap(crossover, [(parents[i-1], parents[i]) for i in range(0, population_size, 2)])
            pool.close()
            pool.join()
            population = [mutation(individual) for individual in population]
            population_size = len(population)
    
    # Find the best individual
    fitness_values = []
    for individual in population:
        fitness_values.append(fitness(individual, distances))
    best_individual = population[np.argmin(fitness_values)]
    
    return best_individual



In [6]:
#Threads
num_threads_2=2
num_threads_4=4
num_threads_8=8

In [7]:
# 2 threads

In [8]:
# 532 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/tsp532/att532.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_2, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_2)

Best Individual: [382, 319, 359, 457, 51, 385, 141, 47, 222, 320, 165, 188, 529, 521, 442, 447, 94, 328, 433, 318, 334, 462, 11, 204, 213, 62, 210, 519, 313, 247, 176, 181, 124, 417, 306, 429, 171, 494, 363, 352, 513, 474, 97, 167, 185, 13, 174, 353, 145, 19, 136, 345, 305, 249, 40, 509, 272, 237, 78, 489, 252, 511, 340, 35, 465, 362, 403, 322, 490, 314, 500, 439, 270, 336, 312, 16, 248, 253, 502, 390, 65, 20, 231, 522, 478, 424, 260, 480, 296, 34, 22, 74, 175, 524, 172, 147, 310, 530, 419, 173, 198, 281, 15, 88, 121, 496, 148, 369, 155, 300, 169, 89, 190, 517, 355, 129, 484, 109, 392, 255, 303, 432, 242, 512, 342, 244, 324, 38, 443, 52, 106, 123, 311, 134, 27, 254, 230, 1, 420, 117, 267, 531, 28, 304, 95, 157, 39, 126, 212, 18, 193, 196, 246, 146, 161, 236, 423, 168, 245, 425, 127, 357, 259, 87, 85, 285, 208, 452, 7, 189, 481, 492, 170, 335, 487, 37, 361, 415, 413, 330, 140, 91, 30, 459, 397, 48, 520, 408, 115, 514, 468, 162, 262, 50, 36, 501, 477, 195, 437, 416, 274, 450, 116, 364, 3

In [9]:
# 1000 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/dsj1000/dsj1000.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_2, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_2)

Best Individual: [99, 556, 402, 411, 10, 214, 572, 51, 341, 32, 431, 786, 462, 129, 136, 586, 874, 221, 222, 602, 901, 688, 705, 697, 144, 904, 995, 751, 561, 747, 364, 972, 223, 577, 877, 190, 458, 15, 438, 754, 560, 597, 460, 551, 406, 771, 903, 718, 685, 89, 938, 158, 852, 945, 678, 467, 756, 296, 837, 865, 736, 567, 937, 428, 307, 919, 912, 794, 910, 299, 381, 977, 719, 859, 169, 768, 880, 953, 98, 793, 52, 726, 950, 161, 998, 604, 533, 605, 616, 775, 730, 608, 27, 973, 209, 272, 444, 172, 207, 26, 817, 798, 440, 828, 996, 958, 521, 813, 257, 463, 777, 631, 66, 211, 618, 822, 818, 684, 230, 70, 324, 962, 310, 527, 40, 766, 842, 115, 6, 54, 149, 907, 634, 963, 514, 672, 931, 871, 164, 727, 292, 366, 195, 483, 318, 695, 46, 796, 725, 339, 430, 960, 947, 503, 887, 393, 930, 305, 4, 67, 293, 488, 781, 11, 266, 93, 541, 860, 723, 936, 838, 809, 630, 668, 150, 125, 983, 853, 589, 841, 952, 116, 873, 615, 905, 664, 88, 823, 325, 633, 275, 497, 72, 179, 949, 306, 956, 579, 701, 519, 906, 9

In [10]:
# 1291 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/data1291/d1291.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_2, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_2)

Best Individual: [788, 620, 466, 984, 320, 1276, 196, 531, 1037, 339, 699, 1211, 654, 421, 769, 465, 55, 883, 125, 862, 1241, 114, 811, 1188, 258, 929, 600, 1199, 527, 571, 888, 1119, 437, 901, 143, 71, 434, 663, 568, 801, 417, 177, 762, 915, 167, 331, 501, 578, 1135, 893, 1192, 452, 900, 1249, 12, 341, 405, 1123, 393, 1106, 722, 632, 233, 379, 36, 710, 515, 1245, 1059, 547, 292, 314, 364, 403, 97, 442, 395, 190, 617, 667, 1031, 892, 170, 448, 188, 61, 1041, 1120, 987, 944, 376, 794, 601, 385, 33, 851, 1265, 528, 1163, 302, 440, 734, 326, 1263, 1153, 251, 787, 949, 219, 1057, 1228, 73, 879, 255, 740, 1243, 497, 499, 246, 111, 842, 438, 786, 318, 1095, 482, 652, 1125, 1161, 492, 1176, 1006, 342, 856, 334, 81, 407, 1079, 269, 1285, 505, 491, 1207, 257, 373, 976, 296, 493, 1244, 274, 698, 582, 644, 418, 999, 67, 330, 992, 1257, 308, 503, 506, 1281, 1116, 362, 198, 189, 472, 875, 783, 729, 707, 1070, 185, 369, 922, 684, 643, 135, 471, 1113, 1264, 184, 1168, 994, 262, 695, 375, 348, 800, 93

In [ ]:
#4 Threads

In [11]:
# 532 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/tsp532/att532.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_4, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_4)

Best Individual: [526, 145, 400, 304, 355, 240, 81, 255, 469, 258, 370, 371, 337, 511, 523, 150, 205, 65, 210, 422, 54, 56, 310, 528, 506, 351, 173, 22, 495, 434, 281, 0, 53, 499, 325, 251, 274, 34, 74, 385, 30, 531, 151, 134, 166, 131, 367, 462, 513, 443, 7, 115, 374, 152, 425, 390, 133, 228, 119, 483, 256, 265, 395, 13, 290, 147, 378, 294, 123, 57, 498, 25, 433, 491, 264, 19, 96, 429, 55, 10, 244, 46, 32, 29, 49, 347, 512, 238, 43, 87, 223, 479, 472, 398, 521, 288, 518, 373, 20, 5, 4, 289, 243, 438, 213, 442, 260, 193, 83, 11, 431, 527, 388, 446, 474, 280, 117, 412, 80, 12, 18, 275, 326, 340, 153, 269, 224, 320, 488, 207, 342, 419, 76, 124, 336, 522, 141, 42, 372, 206, 520, 449, 176, 490, 493, 168, 408, 180, 128, 26, 142, 257, 484, 328, 122, 232, 250, 261, 160, 480, 171, 159, 248, 313, 62, 410, 111, 139, 16, 23, 39, 338, 165, 287, 376, 407, 409, 345, 317, 272, 229, 353, 366, 208, 89, 402, 162, 303, 63, 508, 233, 482, 121, 78, 399, 487, 220, 179, 149, 105, 33, 494, 64, 321, 188, 316, 

In [12]:
# 1000 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/dsj1000/dsj1000.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_4, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_4)

Best Individual: [447, 474, 569, 855, 415, 123, 511, 840, 198, 661, 239, 255, 15, 886, 806, 213, 205, 409, 175, 553, 674, 267, 146, 971, 342, 438, 738, 885, 972, 688, 640, 779, 753, 787, 795, 97, 519, 707, 868, 522, 866, 423, 366, 163, 251, 228, 421, 397, 442, 695, 589, 248, 777, 879, 706, 612, 649, 373, 111, 504, 74, 109, 473, 902, 709, 563, 676, 755, 861, 25, 535, 27, 396, 860, 151, 477, 419, 685, 602, 627, 936, 249, 527, 558, 39, 137, 152, 206, 21, 717, 819, 686, 150, 9, 842, 344, 245, 271, 702, 909, 827, 364, 544, 253, 937, 192, 393, 962, 797, 727, 50, 801, 991, 54, 375, 468, 774, 183, 200, 633, 66, 60, 24, 197, 328, 190, 758, 230, 854, 141, 993, 639, 721, 355, 372, 16, 867, 735, 905, 850, 656, 919, 318, 236, 284, 826, 549, 46, 907, 584, 404, 278, 222, 424, 977, 320, 719, 575, 18, 322, 437, 414, 523, 431, 269, 739, 900, 416, 947, 19, 153, 105, 724, 225, 20, 915, 445, 697, 560, 515, 162, 844, 77, 507, 180, 374, 56, 890, 439, 576, 68, 365, 618, 772, 898, 334, 401, 462, 259, 276, 582,

In [13]:
# 1291 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/data1291/d1291.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_4, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_4)

Best Individual: [1256, 1167, 103, 606, 529, 1020, 338, 935, 151, 869, 37, 891, 236, 763, 518, 850, 833, 559, 107, 208, 416, 419, 448, 1152, 951, 1290, 10, 1269, 27, 35, 17, 98, 587, 204, 372, 658, 347, 1246, 645, 1242, 955, 1027, 920, 803, 29, 976, 974, 114, 1005, 88, 1220, 846, 1182, 882, 608, 340, 1212, 553, 858, 1181, 651, 842, 1266, 69, 762, 13, 1188, 81, 756, 1260, 691, 963, 39, 129, 574, 86, 523, 599, 500, 62, 202, 137, 195, 544, 213, 614, 684, 28, 281, 79, 765, 1221, 670, 61, 427, 766, 123, 473, 985, 11, 1129, 1257, 51, 159, 831, 1003, 937, 184, 1076, 1022, 1141, 366, 386, 1110, 811, 117, 229, 948, 345, 932, 596, 452, 178, 1189, 1105, 875, 476, 1096, 748, 130, 839, 343, 771, 880, 754, 502, 703, 197, 568, 256, 934, 110, 520, 484, 261, 52, 562, 1201, 1253, 902, 1052, 297, 919, 1078, 1132, 481, 150, 613, 870, 581, 1001, 404, 443, 284, 1178, 849, 961, 966, 85, 979, 1111, 1106, 550, 890, 47, 1282, 716, 1210, 1255, 375, 385, 764, 797, 1171, 349, 438, 565, 845, 1126, 1273, 1249, 741, 

In [ ]:
# 8 Threads

In [14]:
# 532 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/tsp532/att532.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_8, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_8)

Best Individual: [151, 125, 214, 76, 498, 153, 436, 281, 375, 276, 462, 167, 171, 442, 285, 258, 35, 217, 215, 36, 421, 518, 191, 328, 115, 313, 450, 293, 208, 156, 32, 155, 283, 278, 67, 317, 314, 391, 199, 359, 158, 15, 37, 345, 347, 137, 296, 476, 342, 200, 451, 18, 429, 455, 79, 223, 513, 17, 493, 480, 470, 40, 449, 122, 234, 152, 386, 242, 106, 249, 141, 467, 320, 323, 443, 28, 269, 408, 53, 524, 377, 195, 484, 213, 222, 241, 446, 180, 497, 490, 186, 516, 472, 303, 279, 147, 105, 254, 387, 221, 219, 425, 118, 371, 240, 485, 88, 270, 410, 297, 205, 134, 112, 98, 398, 477, 39, 309, 424, 390, 0, 426, 255, 526, 465, 58, 110, 244, 66, 453, 372, 401, 70, 389, 57, 103, 107, 97, 531, 478, 396, 227, 298, 44, 20, 403, 311, 55, 124, 346, 378, 260, 119, 289, 233, 427, 73, 416, 43, 245, 483, 288, 343, 414, 486, 334, 264, 266, 198, 290, 237, 348, 457, 319, 29, 85, 515, 201, 420, 22, 116, 194, 433, 300, 12, 382, 528, 415, 248, 51, 127, 360, 204, 310, 479, 418, 407, 181, 503, 61, 502, 92, 136, 10

In [5]:
# 1000 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/dsj1000/dsj1000.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_8, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_8)

Best Individual: [20, 146, 736, 676, 544, 218, 186, 318, 85, 158, 632, 304, 630, 574, 249, 614, 513, 1, 865, 907, 556, 335, 120, 332, 73, 251, 184, 466, 852, 411, 222, 977, 916, 702, 959, 919, 272, 257, 159, 407, 461, 415, 421, 813, 343, 92, 200, 540, 305, 850, 502, 56, 88, 619, 533, 206, 740, 147, 909, 50, 114, 602, 981, 14, 806, 253, 741, 866, 578, 284, 102, 708, 566, 261, 851, 465, 311, 123, 163, 516, 561, 673, 175, 49, 668, 121, 669, 870, 35, 787, 422, 168, 493, 336, 799, 612, 628, 18, 231, 220, 258, 837, 859, 232, 788, 995, 714, 494, 474, 986, 464, 119, 177, 756, 901, 340, 818, 211, 254, 781, 463, 312, 582, 812, 913, 250, 12, 324, 822, 692, 609, 117, 406, 151, 829, 885, 131, 173, 537, 390, 113, 743, 34, 111, 710, 230, 255, 579, 576, 149, 139, 133, 590, 760, 833, 286, 514, 970, 448, 287, 905, 883, 567, 773, 486, 940, 593, 9, 944, 190, 983, 545, 846, 633, 86, 127, 303, 564, 471, 872, 475, 3, 385, 759, 836, 97, 197, 13, 845, 869, 824, 860, 360, 112, 521, 61, 515, 450, 196, 699, 310, 

In [7]:
# 1291 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/data1291/d1291.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, num_threads_8, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('size:', len(best_individual))
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))
print('Number threads:', num_threads_8)

Best Individual: [768, 668, 96, 571, 210, 1143, 536, 90, 781, 1070, 281, 159, 367, 1288, 720, 603, 371, 274, 1107, 1170, 1200, 1228, 522, 1182, 881, 196, 1135, 149, 203, 66, 1277, 632, 11, 555, 590, 299, 1050, 535, 169, 426, 568, 639, 983, 982, 1071, 28, 890, 1227, 915, 709, 561, 882, 1218, 514, 1142, 971, 946, 1223, 893, 500, 264, 221, 1109, 662, 1183, 870, 572, 526, 958, 703, 494, 1057, 655, 167, 492, 262, 566, 1133, 877, 907, 1282, 20, 193, 848, 251, 171, 241, 610, 501, 545, 682, 486, 144, 1289, 22, 1248, 472, 180, 231, 693, 176, 823, 968, 317, 934, 1072, 1260, 348, 1270, 1127, 352, 960, 434, 1193, 1139, 458, 1079, 519, 1030, 811, 900, 1116, 234, 473, 989, 154, 439, 78, 402, 1025, 366, 282, 294, 607, 309, 247, 1279, 945, 845, 498, 835, 91, 909, 119, 1058, 120, 764, 762, 643, 442, 1123, 749, 1192, 453, 766, 1039, 799, 969, 803, 673, 741, 836, 1221, 604, 807, 195, 233, 1266, 435, 821, 758, 1176, 695, 712, 834, 89, 722, 1230, 127, 1147, 496, 1031, 224, 690, 59, 592, 531, 273, 19, 73, 9